In [16]:
import torch

d = torch.load("./data/preprocessed/01052500.pt")
d["weather_daymet"].shape
d["node_attrs"].shape

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13380\3981302650.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d = torch.load("./data/preprocessed/01052500.pt"

torch.Size([8, 15])

In [14]:
d = torch.load("./runs/global/checkpoint/checkpoint_epoch_30.pt")
d["model_state_dict"].keys()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13380\68537307.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d = torch.load("./runs/global/checkpoint/checkpoin

odict_keys(['input.weight', 'input.bias', 'static_emb.0.weight', 'static_emb.0.bias', 'static_emb.1.weight', 'static_emb.1.bias', 'tcn_a.0.weight', 'tcn_a.0.bias', 'tcn_a.1.weight', 'tcn_a.1.bias', 'tcn_a.2.weight', 'tcn_a.2.bias', 'tcn_a.3.weight', 'tcn_a.3.bias', 'tcn_b.0.weight', 'tcn_b.0.bias', 'tcn_b.1.weight', 'tcn_b.1.bias', 'tcn_b.2.weight', 'tcn_b.2.bias', 'tcn_b.3.weight', 'tcn_b.3.bias', 'gat.0.att_src', 'gat.0.att_dst', 'gat.0.bias', 'gat.0.lin.weight', 'gat.1.att_src', 'gat.1.att_dst', 'gat.1.bias', 'gat.1.lin.weight', 'gat.2.att_src', 'gat.2.att_dst', 'gat.2.bias', 'gat.2.lin.weight', 'gat.3.att_src', 'gat.3.att_dst', 'gat.3.bias', 'gat.3.lin.weight', 'bn.0.weight', 'bn.0.bias', 'bn.0.running_mean', 'bn.0.running_var', 'bn.0.num_batches_tracked', 'bn.1.weight', 'bn.1.bias', 'bn.1.running_mean', 'bn.1.running_var', 'bn.1.num_batches_tracked', 'bn.2.weight', 'bn.2.bias', 'bn.2.running_mean', 'bn.2.running_var', 'bn.2.num_batches_tracked', 'bn.3.weight', 'bn.3.bias', 'bn.3.r

In [ ]:
from engine import Engine
import torch
e = Engine(config_path="./config/test.yaml")
train_batches = e._get_batches()
batches, batch_idxs = e._get_batches()
for batch, graph_data in batches:
    node_attrs = graph_data['node_attrs']
    edge_index = graph_data['edge_index']
    edge_attr = graph_data['edge_attr']
    outlet_idx = graph_data['idx']
    gauge_id = graph_data['gauge_id']

    num_nodes = node_attrs.shape[0]
    obs_streamflow = batch['obs_streamflow']
    sim_streamflow = batch['sim_streamflow'].unsqueeze(-1)
    target_obs = batch['target_obs']
    target_sim = batch['target_sim']
    if batch['weather'].dim() == 3:
        weather = batch['weather'].unsqueeze(2).expand(-1, -1, num_nodes, -1)
    elif batch['weather'].dim() == 4:
        weather = batch['weather']
    else:
        raise ValueError
    dynamic = torch.cat([weather, sim_streamflow], dim=-1)
    break

e.model.to("cpu")
output = e.model(dynamic, node_attrs, edge_index, edge_attr)
print(output.shape)

GraphWaveNet(
  (input): Conv2d(7, 16, kernel_size=(1, 1), stride=(1, 1))
  (static_emb): Sequential(
    (0): Linear(in_features=15, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=16, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.3, inplace=False)
  )
  (tcn_a): ModuleList(
    (0): Conv2d(16, 16, kernel_size=(1, 2), stride=(1, 1))
    (1): Conv2d(16, 16, kernel_size=(1, 2), stride=(1, 1), dilation=(2, 2))
    (2): Conv2d(16, 16, kernel_size=(1, 2), stride=(1, 1), dilation=(4, 4))
    (3): Conv2d(16, 16, kernel_size=(1, 2), stride=(1, 1), dilation=(8, 8))
  )
  (tcn_b): ModuleList(
    (0): Conv2d(16, 16, kernel_size=(1, 2), stride=(1, 1))
    (1): Conv2d(16, 16, kernel_size=(1, 2), stride=(1, 1), dilation=(2, 2))
    (2): Conv2d(16, 16, kernel_size=(1, 2), stride=(1, 1), dilation=(4, 4))
    (3): Conv2d(16, 16, kernel_size=(1, 2), stride=(1, 1), dilation=(8, 8))
  )
  (gat): ModuleList(
    (0-3): 4 x GATConv(16, 16, heads=4)
  )
  (bn): Module

c:\Users\82105\GATWN\GATWN\dataset.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(os.path.join(self.data_dir, f"{self.gauge_id}.pt"))
c:\Users\82105\

torch.Size([256, 7, 8, 21])
torch.Size([256, 16, 8, 21]) torch.Size([8, 16])


RuntimeError: The size of tensor a (21) must match the size of tensor b (16) at non-singleton dimension 3

In [27]:
dynamic.shape, node_attrs.shape, edge_index.shape, edge_attr.shape

(torch.Size([256, 21, 8, 7]),
 torch.Size([8, 15]),
 torch.Size([2, 7]),
 torch.Size([7, 3]))